In [0]:
corpus = ['king is a strong man', 
          'queen is a wise woman', 
          'boy is a young man',
          'girl is a young woman',
          'prince is a young king',
          'princess is a young queen',
          'man is strong', 
          'woman is pretty',
          'prince is a boy will be king',
          'princess is a girl will be queen']

In [0]:
#불용어 제거
def remove_stop_words(corpus):
    stop_words = ['is', 'a', 'will', 'be']
    results = []
    for text in corpus:
        tmp = text.split(' ')
        for stop_word in stop_words:
            if stop_word in tmp:
                tmp.remove(stop_word)
        results.append(" ".join(tmp))
    
    return results

In [0]:
corpus = remove_stop_words(corpus)
corpus

In [0]:
words = []
for text in corpus:
    for word in text.split(' '):
        words.append(word)

words = set(words)

### word2vec table 생성

In [0]:
word2int = {}

for i,word in enumerate(words):
    word2int[word] = i

sentences = []
for sentence in corpus:
    sentences.append(sentence.split())
    
# 앞뒤 몇개 단어를 가져올거니?
WINDOW_SIZE = 2

data = []
for sentence in sentences:
    for idx, word in enumerate(sentence):
        for neighbor in sentence[max(idx - WINDOW_SIZE, 0) : min(idx + WINDOW_SIZE, len(sentence)) + 1] : 
            if neighbor != word:
                data.append([word, neighbor])

In [0]:
import pandas as pd
for text in corpus:
    print(text)

df = pd.DataFrame(data, columns = ['input', 'label'])

In [0]:
from nltk.tokenize import word_tokenize, sent_tokenize

In [0]:
result = [word_tokenize(text) for text in corpus]

In [0]:
from gensim.models import Word2Vec
#size=w1,w2...의 개수
model = Word2Vec(sentences=result, size=2, window=2, min_count=1, workers=4, sg=1)

In [0]:
model_result = model.wv.most_similar("man")
print(model_result)

In [0]:
#모델저장하기
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('eng_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") #불러오기

In [0]:
model_result = loaded_model.most_similar("man")
print(model_result)

### kakao arena에 해보기

In [0]:
import pickle
with open('train_genre.pickle', 'rb') as f:
    train = pickle.load(f)

In [0]:
import pandas as pd
train = pd.DataFrame(train)

In [0]:
import sys
text = open('output.txt','w')
i=0
while i < len(corpus):
    text.write(corpus[i]+'\n')
    print(corpus[i])
    i+=1
text.close()

In [0]:
!pip install sentencepiece

In [0]:
import sentencepiece as spm
spm.SentencePieceTrainer.Train('--input=output.txt --model_prefix=m --vocab_size=31')

In [0]:
sp = spm.SentencePieceProcessor()
sp.Load("m.model")
#kkma([train.plylst_title[1],(train.tags[1])])

In [0]:
# tag & playlist 하나로 합치고, cleansing 해서 txt파일로 저장

def clean_str(text):
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '[^\w\s]'         # 특수기호제거
    text = re.sub(pattern=pattern, repl='', string=text)
    return text  

i=0
import sys
text = open('output.txt','w')

while i < len(train):
    mylist = [[train.plylst_title[i]],train.tags[i]]
    clean = clean_str(' '.join(sum(mylist,[])))
    text.write(clean+'\n')
    print(clean)
    i+=1
text.close()


In [0]:
with open('output.txt') as f:
    data = f.read().splitlines() 

In [0]:
data

In [0]:
result = [word_tokenize(text) for text in data]
model = Word2Vec(sentences=result, size=2, window=2, min_count=10, workers=4, sg=1)
model_result = model.wv.most_similar("man")
print(model_result)

In [0]:
model_result = model.wv.most_similar("컨트리")

In [0]:
print(model_result)